### 파이프라인(Pipeline) 

- 분류(Classification) 분석을 진행할 때 파이프라인(Pipeline)을 활용하면 
- 전처리 → 특징 변환 → 모델 학습 → 평가 과정을 깔끔하게 묶어서 처리할 수 있다.

## 1. 기본 예시

### 1-1 데이터 준비

In [1]:
from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split

# 데이터 로드
wine = load_wine()
X = wine.data
y = wine.target

# 학습용/테스트용 분리
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.2, 
                                                    random_state=42)


### 1-2. 파이프라인 구성

- 분류모델 (로지스틱회귀)

In [3]:
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression

# 파이프라인 정의
pipe = Pipeline([
    ("imputer", SimpleImputer(strategy="mean")),   # 결측치 처리
    ("scaler", StandardScaler()),                  # 스케일링
    ("model", LogisticRegression(max_iter=1000))   # 분류 모델
])


### 1-3. 모델 학습 및 예측

In [4]:
pipe.fit(X_train, y_train)
y_pred = pipe.predict(X_test)

### 1-4. 성능 평가

In [5]:
from sklearn.metrics import classification_report, confusion_matrix

print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))


[[14  0  0]
 [ 0 14  0]
 [ 0  0  8]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        14
           1       1.00      1.00      1.00        14
           2       1.00      1.00      1.00         8

    accuracy                           1.00        36
   macro avg       1.00      1.00      1.00        36
weighted avg       1.00      1.00      1.00        36



### 1-5.파이프라인 + GridSearchCV

In [ ]:
# 파이프라인 내부 모델 파라미터는 "단계이름__파라미터명" 형식으로 접근

# 전처리 하이퍼파라미터(예: 스케일링 방식)도 동일한 방식으로 튜닝 가능

In [6]:
from sklearn.model_selection import GridSearchCV

# 탐색할 하이퍼파라미터 정의
param_grid = {
    "model__C": [0.01, 0.1, 1, 10],
    "model__penalty": ["l1", "l2"],
    "model__solver": ["liblinear"]
}

# 그리드서치 객체 생성
grid = GridSearchCV(pipe, param_grid, cv=5, scoring="accuracy")
grid.fit(X_train, y_train)

print("최적 파라미터:", grid.best_params_)
print("최적 점수:", grid.best_score_)


최적 파라미터: {'model__C': 1, 'model__penalty': 'l2', 'model__solver': 'liblinear'}
최적 점수: 0.9859605911330049


### 1-6. ColumnTransformer와 함께 쓰기

In [ ]:
# 실무에서는 수치형 + 범주형 변수가 섞여 있는 경우--> 이럴 땐 ColumnTransformer를 파이프라인에 통합하면 편리합니다.

In [7]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

# 예시: 0~2번 컬럼은 수치형, 3~4번 컬럼은 범주형이라고 가정
preprocessor = ColumnTransformer([
    ("num", StandardScaler(), [0, 1, 2]),
    ("cat", OneHotEncoder(), [3, 4])
])

pipe = Pipeline([
    ("preprocessing", preprocessor),
    ("model", LogisticRegression(max_iter=1000))
])


## 2. LogisticRegression / RandomForest / XGBoost / SVM 네 가지 분류기를 각각 GridSearchCV

In [ ]:
# ColumnTransformer:
# 수치형(num_*) → SimpleImputer(median) → StandardScaler
# 범주형(cat_0, cat_1) → SimpleImputer(most_frequent) → OneHotEncoder(handle_unknown="ignore")
# Pipeline ("preprocess", preprocessor) → ("model", <Classifier>)
# GridSearchCV: 전처리 결과를 훈련 폴드에서만 학습에 사용하므로 데이터 누수 방지
# 공통 cv=StratifiedKFold(5), scoring="f1_macro"
# 평가: Accuracy, F1(macro), Confusion Matrix, Classification Report

In [10]:
# ===============================
# 분류 파이프라인 4종 비교 스크립트
# Logistic / RandomForest / XGBoost / SVM
# ===============================

# 1. 라이브러리 
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

# XGBoost (pip install xgboost)
from xgboost import XGBClassifier


# 2. 데이터 생성: 수치형 + 범주형 혼합 (자급자족) 
# 분류(Classification)용 샘플 데이터 생성 함수
from sklearn.datasets import make_classification

np.random.seed(42)  # 난수 생성기의 **시드(seed)**를 고정하는 역할
X_num, y = make_classification(
    n_samples=1200,     # 총 샘플 수 (1200개의 데이터 포인트 생성)
    n_features=6,    # 총 샘플 수 (1200개의 데이터 포인트 생성)
    n_informative=4,  # 실제로 분류에 기여하는 중요한 특징 수
    n_redundant=0,   # informative 특징을 선형결합해서 만든 중복 특징 수
    n_repeated=0,  # informative/redundant 특징을 그대로 복사한 특징 수
    n_classes=2,  # 클래스 개수 → 이진 분류
    class_sep=1.2, # 클래스 간 중심 간격 → 값이 클수록 클래스가 더 잘 분리됨
    weights=[0.6, 0.4],   # 클래스별 샘플 비율 지정  → 클래스 0: 60%, 클래스 1: 40%
    random_state=42 # 난수 시드 → 항상 동일한 데이터셋 생성
)


# NumPy 배열을 pandas DataFrame으로 변환하면서, 자동으로 컬럼명까지 생성하는 부분 : 수치형 6개를 DataFrame으로
# X_num : NumPy 배열 → shape = (1200, 6)
# → make_classification()으로 생성한 1200개 샘플 × 6개 특징 데이터
# pd.DataFrame() : NumPy 배열을 pandas DataFrame으로 변환
# columns=[...] : 컬럼명을 자동 생성하여 지정
# X_num.shape[1] → X_num의 열 개수, 즉 특징(feature) 수

df = pd.DataFrame(X_num, columns=[f"num_{i}" for i in range(X_num.shape[1])])

# 범주형 특성 2개를 추가 (의미 있는 신호 약간 부여)
# cat_0: 3개 레벨(A/B/C), y=1일 때 B 비율 조금 높게
cat0 = np.where(np.random.rand(len(y)) + 0.15*(y==1) > 0.67, "B",
        np.where(np.random.rand(len(y)) > 0.5, "A", "C"))

# cat_1: 4개 레벨, 무작위
cat1_levels = np.array(["X","Y","Z","W"])
cat1 = np.random.choice(cat1_levels, size=len(y), p=[0.25,0.25,0.25,0.25])

df["cat_0"] = cat0
df["cat_1"] = cat1

# 목표변수
target = pd.Series(y, name="target")

# 학습/검증 분할
X_train, X_test, y_train, y_test = train_test_split(
    df, target, test_size=0.2, random_state=42, stratify=target
)

# 3. 전처리 파이프라인 구성 -----
num_cols = [c for c in df.columns if c.startswith("num_")]
cat_cols = ["cat_0", "cat_1"]

numeric_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(handle_unknown="ignore"))
])

# ColumnTransformer를 사용해 수치형(numeric)과 범주형(categorical) 데이터를 각각 다른 전처리 방식으로 처리하기 위한 설정
preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, num_cols),
        ("cat", categorical_transformer, cat_cols)
    ]
)

# 4. 네 가지 모델용 파이프라인 정의 -----
pipe_logit = Pipeline([
    ("preprocess", preprocessor),
    ("model", LogisticRegression(max_iter=2000))
])

pipe_rf = Pipeline([
    ("preprocess", preprocessor),
    ("model", RandomForestClassifier(random_state=42))
])

pipe_xgb = Pipeline([
    ("preprocess", preprocessor),
    ("model", XGBClassifier(
        objective="binary:logistic",
        eval_metric="logloss",
        tree_method="hist",      # CPU 빠른 학습
        random_state=42,
        n_estimators=300
    ))
])

pipe_svm = Pipeline([
    ("preprocess", preprocessor),
    ("model", SVC(probability=True, random_state=42))
])

# 5. 하이퍼파라미터 그리드 -----
param_logit = {
    "model__C": [0.1, 1.0, 10.0],
    "model__penalty": ["l2"],          # (liblinear일 경우 l1도 가능)
    "model__solver": ["lbfgs"]         # 다수 특성에 안정적
}

param_rf = {
    "model__n_estimators": [200, 400],
    "model__max_depth": [None, 8, 12],
    "model__min_samples_split": [2, 5],
    "model__min_samples_leaf": [1, 2]
}

param_xgb = {
    "model__max_depth": [3, 5],
    "model__learning_rate": [0.05, 0.1],
    "model__subsample": [0.8, 1.0],
    "model__colsample_bytree": [0.8, 1.0]
}

param_svm = {
    "model__C": [0.5, 1.0, 2.0],
    "model__kernel": ["rbf", "poly"],
    "model__gamma": ["scale", "auto"]
}

models = {
    "LogisticRegression": (pipe_logit, param_logit),
    "RandomForest": (pipe_rf, param_rf),
    "XGBoost": (pipe_xgb, param_xgb),
    "SVM": (pipe_svm, param_svm)
}

# 6. 공통 CV 설정
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

def run_grid_search(name, pipe, param_grid):
    grid = GridSearchCV(
        estimator=pipe,
        param_grid=param_grid,
        cv=cv,
        scoring="f1_macro",      # 불균형 대비 종합성
        n_jobs=-1,
        verbose=0
    )
    grid.fit(X_train, y_train)
    best = grid.best_estimator_
    y_pred = best.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    f1m = f1_score(y_test, y_pred, average="macro")

    print("="*70)
    print(f"[{name}]")
    print("Best params:", grid.best_params_)
    print(f"CV Best F1(macro): {grid.best_score_:.4f}")
    print(f"Test Accuracy   : {acc:.4f}")
    print(f"Test F1 (macro) : {f1m:.4f}")
    print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))
    print("\nClassification Report:\n", classification_report(y_test, y_pred, digits=4))

# 7. 실행 -----
for name, (pipe, params) in models.items():
    run_grid_search(name, pipe, params)


[LogisticRegression]
Best params: {'model__C': 0.1, 'model__penalty': 'l2', 'model__solver': 'lbfgs'}
CV Best F1(macro): 0.7904
Test Accuracy   : 0.8125
Test F1 (macro) : 0.7945

Confusion Matrix:
 [[133  11]
 [ 34  62]]

Classification Report:
               precision    recall  f1-score   support

           0     0.7964    0.9236    0.8553       144
           1     0.8493    0.6458    0.7337        96

    accuracy                         0.8125       240
   macro avg     0.8229    0.7847    0.7945       240
weighted avg     0.8176    0.8125    0.8067       240

[RandomForest]
Best params: {'model__max_depth': None, 'model__min_samples_leaf': 1, 'model__min_samples_split': 2, 'model__n_estimators': 400}
CV Best F1(macro): 0.9206
Test Accuracy   : 0.9250
Test F1 (macro) : 0.9203

Confusion Matrix:
 [[140   4]
 [ 14  82]]

Classification Report:
               precision    recall  f1-score   support

           0     0.9091    0.9722    0.9396       144
           1     0.9535    0.8

- ColumnTransformer + Pipeline을 공통 전처리로 묶고, 
- LogisticRegression / RandomForest / XGBoost / SVM 네 모델을 GridSearchCV로 튜닝·평가

In [14]:
# ============================================================
# 분류 파이프라인 4종 비교 (ColumnTransformer + Pipeline + GridSearchCV)
# ============================================================
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score, f1_score, classification_report, confusion_matrix

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

# (선택) XGBoost: 미설치면 자동 건너뜀
has_xgb = True
try:
    from xgboost import XGBClassifier
except Exception:
    has_xgb = False

# 1) ----- 데이터 만들기: 수치형 6개 + 범주형 2개 -----
from sklearn.datasets import make_classification
np.random.seed(42)
X_num, y = make_classification(
    n_samples=1200, n_features=6, n_informative=4, n_redundant=0,
    n_repeated=0, n_classes=2, class_sep=1.2, weights=[0.6, 0.4],
    random_state=42
)
df = pd.DataFrame(X_num, columns=[f"num_{i}" for i in range(X_num.shape[1])])

# 범주형 특성 2개 (간단 신호/노이즈)
cat0 = np.where(np.random.rand(len(y)) + 0.15*(y==1) > 0.67, "B",
        np.where(np.random.rand(len(y)) > 0.5, "A", "C"))
cat1 = np.random.choice(["X","Y","Z","W"], size=len(y))
df["cat_0"] = cat0
df["cat_1"] = cat1
target = pd.Series(y, name="target")

X_train, X_test, y_train, y_test = train_test_split(
    df, target, test_size=0.2, stratify=target, random_state=42
)

# 2) ----- 전처리: 수치형/범주형 분리 처리 -----
num_cols = [c for c in df.columns if c.startswith("num_")]
cat_cols = ["cat_0", "cat_1"]

numeric_transformer = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler()),
])

categorical_transformer = Pipeline([
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(handle_unknown="ignore")),
])

preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, num_cols),
        ("cat", categorical_transformer, cat_cols),
    ]
)

# 3) ----- 모델 파이프라인들 -----
pipe_logit = Pipeline([("preprocess", preprocessor),
                       ("model", LogisticRegression(max_iter=2000))])

pipe_rf    = Pipeline([("preprocess", preprocessor),
                       ("model", RandomForestClassifier(random_state=42))])

pipe_svm   = Pipeline([("preprocess", preprocessor),
                       ("model", SVC(probability=True, random_state=42))])

if has_xgb:
    pipe_xgb = Pipeline([("preprocess", preprocessor),
                         ("model", XGBClassifier(
                             objective="binary:logistic",
                             eval_metric="logloss",
                             tree_method="hist",
                             random_state=42,
                         ))])

# 4) ----- 하이퍼파라미터 그리드 -----
param_logit = {
    "model__C": [0.1, 1.0, 10.0],
    "model__penalty": ["l2"],
    "model__solver": ["lbfgs"],
}
param_rf = {
    "model__n_estimators": [200, 400],
    "model__max_depth": [None, 10],
    "model__min_samples_split": [2, 5],
    "model__min_samples_leaf": [1, 2],
}
param_svm = {
    "model__C": [0.5, 1.0, 2.0],
    "model__kernel": ["rbf", "poly"],
    "model__gamma": ["scale", "auto"],
}
if has_xgb:
    param_xgb = {
        "model__max_depth": [3, 5],
        "model__learning_rate": [0.05, 0.1],
        "model__subsample": [0.8, 1.0],
        "model__colsample_bytree": [0.8, 1.0],
        "model__n_estimators": [200, 400],
    }

models = [
    ("LogisticRegression", pipe_logit, param_logit),
    ("RandomForest",       pipe_rf,   param_rf),
    ("SVM",                pipe_svm,  param_svm),
]
if has_xgb:
    models.append(("XGBoost", pipe_xgb, param_xgb))

# 5) ----- 공통 CV/스코어 -----
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
SCORING = "f1_macro"  # 불균형 대응 종합성

def run_grid(name, pipe, grid):
    gs = GridSearchCV(pipe, grid, cv=cv, scoring=SCORING, n_jobs=-1, verbose=0)
    gs.fit(X_train, y_train)
    best = gs.best_estimator_
    y_pred = best.predict(X_test)
    acc  = accuracy_score(y_test, y_pred)
    f1m  = f1_score(y_test, y_pred, average="macro")

    print("="*72)
    print(f"[{name}]")
    print("Best params:", gs.best_params_)
    print(f"CV Best {SCORING}: {gs.best_score_:.4f}")
    print(f"Test Accuracy   : {acc:.4f}")
    print(f"Test F1 (macro) : {f1m:.4f}")
    print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))
    print("\nClassification Report:\n", classification_report(y_test, y_pred, digits=4))

for name, pipe, grid in models:
    run_grid(name, pipe, grid)

if not has_xgb:
    print("\n[알림] xgboost 미설치로 XGBoost 실험은 건너뛰었습니다. (pip install xgboost)")


[LogisticRegression]
Best params: {'model__C': 1.0, 'model__penalty': 'l2', 'model__solver': 'lbfgs'}
CV Best f1_macro: 0.7885
Test Accuracy   : 0.8125
Test F1 (macro) : 0.7967

Confusion Matrix:
 [[131  13]
 [ 32  64]]

Classification Report:
               precision    recall  f1-score   support

           0     0.8037    0.9097    0.8534       144
           1     0.8312    0.6667    0.7399        96

    accuracy                         0.8125       240
   macro avg     0.8174    0.7882    0.7967       240
weighted avg     0.8147    0.8125    0.8080       240

[RandomForest]
Best params: {'model__max_depth': None, 'model__min_samples_leaf': 1, 'model__min_samples_split': 5, 'model__n_estimators': 400}
CV Best f1_macro: 0.9204
Test Accuracy   : 0.9167
Test F1 (macro) : 0.9111

Confusion Matrix:
 [[140   4]
 [ 16  80]]

Classification Report:
               precision    recall  f1-score   support

           0     0.8974    0.9722    0.9333       144
           1     0.9524    0.833